In [1]:
import cv2
import os
import numpy as np
import matplotlib.pyplot as plt  
from ultralytics import YOLO
import math
import time 

In [2]:
# --- Load the Saved Model YOLOv8 ---
model_path = "runs/detect/train10/weights/best.pt"  # Path to the saved model
model = YOLO(model_path)  # Load for YOLOv8

In [3]:
# Define the path to the directory containing the images
image_directory = r"C:\Users\Vahid\Documents\Jupyter\custom_yolov8\dataset_for_yolo\images\train"  

# List all image files in the directory  
image_files = [f for f in os.listdir(image_directory) if f.endswith(('.png', '.jpg', '.jpeg', '.bmp'))]  

output_path = "C:/Users/Vahid/Documents/Jupyter/custom_yolov8/result.jpg"

In [4]:
def detect_obj (image):
    """This function gets an image, passes it through the Yolo model to detect the pill box
    The function returns the image and the coordiantions of the frame containing the detected box
    Args:
        image: RGB image
        
    Returns:
        image: RGB image with the detected object
        x1, y1, x2, y2: rectangle around the detected pill box
    """
    
    # Run Inference
    results = model.predict(source=image, device="cpu")  # YOLOv8

    # Extract Detected Boxes & Draw Rectangles
    for result in results:
        for box in result.boxes:
            # Get coordinates (unnormalized)
            x1, y1, x2, y2 = map(int, box.xyxy[0].tolist())
    
            # Draw a green rectangle (BGR: (0, 255, 0))
            cv2.rectangle(image, (x1, y1), (x2, y2), (0, 255, 0), 2)
    
            # Add label and confidence
            conf = box.conf[0].item()
            cls = box.cls[0].item()
            label = f"{model.names[int(cls)]} {conf:.2f}"
            cv2.putText(image, label, (x1, y1-10), 
            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1)
    return image, x1, y1, x2, y2

In [5]:
def get_box(x1, y1, x2, y2):
    """This function gets the coordinates and returns the ROI cut from the original image
    """
    image2 = original[ y1:y2, x1: x2]
    return image2

In [6]:
def get_mask (image):
    """
    Function get_mask, receives the ROI and extracts a binary mask from the green part of the pill box.
    Args:
        image: RGB image
        
    Returns:
        image: Binary image which is the extracted green zone of the pill box
    """

    image2 = cv2.cvtColor(Box, cv2.COLOR_BGR2RGB)  # Convert to RGB  
    hsv_image = cv2.cvtColor(image2, cv2.COLOR_RGB2HSV)  # Convert to HSV  
    
    # Create a mask for green color  
    lower_green = np.array([40, 100, 50])  
    upper_green = np.array([180, 255, 255])  
    
    # Create a mask of the green areas  
    mask = cv2.inRange(hsv_image, lower_green, upper_green)  
    
    # Apply the mask to get the green part of the image  
    green_part = cv2.bitwise_and(image2, image2, mask=mask)  
    
    # Find edges  
    gray_green_part = cv2.cvtColor(green_part, cv2.COLOR_RGB2GRAY) # Convert the green part to grayscale 
    
    # Threshold the gray image to create a binary image  
    _, binary_image = cv2.threshold(gray_green_part, 1, 255, cv2.THRESH_BINARY)  
    
    # Find contours in the binary image  
    contours, _ = cv2.findContours(binary_image, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)  
    
    # Create an empty mask to draw contours larger than 500 pixels  
    large_contours_mask = np.zeros_like(binary_image)  
    
    # Filter and draw contours that are larger than 500 pixels  
    for contour in contours:  
        if cv2.contourArea(contour) > 500:  
            cv2.drawContours(large_contours_mask, [contour], -1, 255, thickness=cv2.FILLED)  
    
    # The resulting mask is a black-and-white image with the gray areas larger than 500 pixels  
    final_output = cv2.resize(large_contours_mask, (large_contours_mask.shape[1]*2, large_contours_mask.shape[0]*2), interpolation=cv2.INTER_LINEAR) 
    
    return final_output

In [7]:
def detect_edge_defects(binary_img, angle_threshold=5, max_deviation=5, min_edge_length=50):
    """
    Detect horizontal edges and mark defects where points deviate from straight line.
    
    Args:
        binary_img: Input binary image (white object on black background)
        angle_threshold: Max deviation from horizontal (degrees)
        max_deviation: Maximum allowed pixel deviation from straight line
        min_edge_length: Minimum edge length to consider
        
    Returns:
        result_img: Image with defects marked
        defect_report: Dictionary of defect information
    """
    # Create output image
    result_img = cv2.cvtColor(binary_img, cv2.COLOR_GRAY2BGR)
    defect_report = {}
    
    # Find contours
    contours, _ = cv2.findContours(binary_img, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    if not contours:
        print("No contours found!")
        return result_img, defect_report
    
    # Get largest contour
    main_contour = max(contours, key=cv2.contourArea)
    
    # Approximate contour to polygon
    epsilon = 0.03 * cv2.arcLength(main_contour, True) # epsilon is maximum distance from contour to approximated contour
    approx = cv2.approxPolyDP(main_contour, epsilon, True) # approxPolyDP approximates a contour shape to another shape with less number of vertices
    
    for edge_idx in range(len(approx)):
        pt1 = approx[edge_idx][0]
        pt2 = approx[(edge_idx+1) % len(approx)][0]
        
        # Calculate edge length and skip short edges
        edge_length = np.linalg.norm(pt2 - pt1)
        if edge_length < min_edge_length:
            continue
        
        # Calculate angle and check if it is horizontal
        angle = np.degrees(np.arctan2(pt2[1] - pt1[1], pt2[0] - pt1[0]))
        angle = abs(angle % 180)
        if not (angle < angle_threshold or angle > 180 - angle_threshold):
            continue
        
        # Get all contour points for this edge segment
        idx1 = np.where((main_contour[:, 0] == pt1).all(axis=1))[0][0]
        idx2 = np.where((main_contour[:, 0] == pt2).all(axis=1))[0][0]
        
        if idx1 > idx2:
            segment = np.concatenate((main_contour[idx1:], main_contour[:idx2+1]))
        else:
            segment = main_contour[idx1:idx2+1]
        segment = segment.squeeze()
        
        # Calculate point-to-line distances
        line_vec = np.array([pt2[0]-pt1[0], pt2[1]-pt1[1]])
        line_len = np.linalg.norm(line_vec)
        line_vec = line_vec / line_len
        normal_vec = np.array([-line_vec[1], line_vec[0]])
        
        defects = []
        for pt in segment:
            vec = pt - pt1
            normal_distance = abs(np.dot(vec, normal_vec))
            if normal_distance > max_deviation:
                defects.append((tuple(pt), normal_distance))
        
        # Visualize results
        cv2.line(result_img, tuple(pt1), tuple(pt2), (0, 255, 0), 2)  # Green line for edge
        
        if defects:
            defect_report[edge_idx] = {
                'edge_points': (tuple(pt1), tuple(pt2)),
                'defect_count': len(defects),
                'max_deviation': max(d[1] for d in defects),
                'avg_deviation': np.mean([d[1] for d in defects])
            }
            
            # Mark defects in red
            for pt, distance in defects:
                cv2.circle(result_img, pt, 3, (0, 0, 255), -1)
                cv2.putText(result_img, f"{distance:.1f}", pt,
                          cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 0, 255), 1)
    
    return result_img, defect_report


In [ ]:
#    Main    --- Loop through the image files ---

paused = False # Initial setup for pause variable 

for image_file in image_files:  
    # Construct the full image path  
    image_path = os.path.join(image_directory, image_file)  
    
    # Read the image  
    original = cv2.imread(image_path) 
    image = original.copy()  # Keep the original image untouched for future processing

    if image is None:
        print(f"Error: Could not load image at {image_path}")

    image, x1, y1, x2, y2 = detect_obj(image)
    Box = get_box(x1, y1, x2, y2)
    
    final_output = get_mask (Box)

    # Process the image
    result, defects = detect_edge_defects(
        final_output,
        angle_threshold=50,    # Max deviation from horizon (degrees)
        max_deviation=5,       # Max allowed pixel deviation
        min_edge_length=50     # Minimum edge length to consider
    )

    # Print defect report
    print(f"Found {len(defects)} edges with defects")
    for edge_idx, data in defects.items():
        print(f"Edge {edge_idx}:")
        print(f"  - Points: {data['edge_points']}")
        print(f"  - Defects: {data['defect_count']}")
        print(f"  - Max deviation: {data['max_deviation']:.2f}px")
        print(f"  - Avg deviation: {data['avg_deviation']:.2f}px")

    # ---  Display control results  ---
    if (len(defects) != 0):
        text = "NOK"  
        font = cv2.FONT_HERSHEY_SIMPLEX  
        font_scale = 2  
        color = (0, 0, 255)  # Red color in BGR  
        thickness = 4  
        position = (100, 100)  # Position to place the text (x, y)  
    
        # Put the text on the image  
        cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)  
    else:
        text = "OK"  
        font = cv2.FONT_HERSHEY_SIMPLEX  
        font_scale = 2  
        color = (0, 255, 0)  # Red color in BGR  
        thickness = 4  
        position = (100, 100)  # Position to place the text (x, y)  
    
        # Put the text on the image  
        cv2.putText(image, text, position, font, font_scale, color, thickness, cv2.LINE_AA)  

    # Guide text on image window for running/pausing control
    cv2.putText(image, "Press 'P' for pause, 'C' for Continue, 'Q' for quit", 
                (50,500), font, 0.5, (200,200,200), 1, cv2.LINE_AA)  
    #cv2.imshow("Edge Defect Detection", result)  
    cv2.imshow("Detection", image)  
  
    # Implement pause and continue functionality  
    # Wait 2000 milliseconds but allow for the window to be closed by the user 
    while True:  
        key = cv2.waitKey(50)  # Check for a key press every 50 milliseconds  

        # If the 'p' key is pressed, pause  
        if key & 0xFF == ord('p'):  
            paused = True  
            print("Paused. Press 'c' to continue.")  
        
        # If the 'c' key is pressed, continue  
        if key & 0xFF == ord('c'):  
            paused = False  
            print("Continuing...")  
            break  
        
        # If 'q' key is pressed, exit the loop  
        if key & 0xFF == ord('q'):  
            print("Exiting...")  
            cv2.destroyAllWindows()  
            exit()  

        # If not paused, break the inner loop  
        if not paused:  
            break  

    # Wait for 2000 milliseconds (2 seconds)  
    if not paused:  
        cv2.waitKey(2000)  

# Clean up any open windows  
cv2.destroyAllWindows()